In [6]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Sort_Vertices, Bounding_Box, Raster_Centroid, Raster_BBox

In [7]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

In [8]:
image_resolution = 1080
image_path = 'gradient1080.jpg'
weight_image = Image.open(image_path)
res = image_resolution
scale_factor = image_resolution / res

img = Image.open(image_path)
img = weight_image
draw = ImageDraw.Draw(img)


In [9]:
imageArray = np.array(weight_image)
def Weighted_Raster_Centroid(pixels):
    cx = 0
    cy = 0
    total_weight = 0
    for pixel in pixels:
        sample_x = pixel[1]
        sample_y = pixel[0]
        # sampled_value = image.getpixel((sample_x,sample_y))
        sampled_value = imageArray[int(sample_x)][int(sample_y)]

        weight = 1 - np.mean(sampled_value) / 255

        if weight < 0.05:
            weight = 0

        total_weight += weight
        
        cx += pixel[0] * weight
        cy += pixel[1] * weight
    
    if total_weight == 0:
        return False

    cx /= total_weight
    cy /= total_weight
    
    return [cx, cy]

In [12]:
polygon = np.array([[150,100],[750,200],[800,400],[700,800],[500,900],[100,500]])

for point in polygon:
    center_ellipse(point[0],point[1],5,'red')
bbox = Raster_BBox(polygon)
rasterized_polygon = Scanline_Rasterize_Polygon(polygon,bbox,1080,1080)[0]

centroidW = Weighted_Raster_Centroid(rasterized_polygon)
centroid = Raster_Centroid(rasterized_polygon)
for pixel in rasterized_polygon:
    # print(pixel)
    center_rectangle(pixel[0],pixel[1],1,1,'gray')

center_ellipse(centroid[0],centroid[1],10,'green')
center_ellipse(centroidW[0],centroidW[1],10,'white')
img.save('centroid_test.png')